In [1]:
import pandas as pd
import numpy as np
import time
import re
from sklearn.metrics.pairwise import cosine_similarity

def year_interval(filter_value,cols):
    filter_value_1 = filter_value.dropna(subset = cols) #filter_value.sort_values(by=['apply_date'],ascending=True)
    return (filter_value_1['申请日'].max()-filter_value_1['申请日'].min()).days//365+1

def process_ipc(filter_patent):
    ipcs_patent = filter_patent.loc[(filter_patent['专利类型'] == '发明授权') | (filter_patent['专利类型'] == '发明申请'), 'IPC'].str.split(';').explode().str.strip().str.extract('(\w+)/')[0].str[:3]
    return 1.0 - (ipcs_patent.value_counts(normalize=True)**2).sum()

def calculate_metrics(filter_value):
    metrics = {}
    
    # 申请号的唯一数量
    metrics['Application_num'] = filter_value['申请号'].nunique()
    
    # 去重后的 IPC 主分类数量
    metrics['IPC主分类_去重'] = filter_value['IPC主分类'].dropna().map(lambda x: x[:3]).nunique()
    
    # IPC 主分类的平均值
    metrics['IPC主分类_平均'] = metrics['IPC主分类_去重'] / year_interval(filter_value, 'IPC主分类')

    # 计算 IPC 的去重数量和平均值
    ipcs = filter_value['IPC'].str.split(';').explode().str.strip()
    metrics['IPC_个数_去重'] = ipcs.nunique()
    metrics['IPC_个数_平均'] = metrics['IPC_个数_去重'] / year_interval(filter_value, 'IPC')

    # 被引证次数和引证数量
    metrics['被引证次数'] = filter_value['被引证次数'].sum()
    metrics['被引证次数_平均'] = filter_value['被引证次数'].mean()
    metrics['引证数量'] = filter_value['引证次数'].sum()
    
    # 引证专利的去重数量和平均值
    citing = filter_value['引证专利'].str.split(';').explode().str.strip()
    metrics['引证数量_去重'] = citing.nunique()
    metrics['引证数量_平均'] = metrics['引证数量_去重'] / year_interval(filter_value, '引证专利')

    # 简单同族专利相关统计
    metrics['简单同族个数'] = filter_value['简单同族个数'].sum()
    cognate = filter_value['简单同族'].str.split(';').explode().str.strip()
    metrics['简单同族_去重'] = cognate.nunique()
    metrics['简单同族美日欧_比例'] = cognate.str[:2].isin(['US', 'JP', 'EP']).sum() / cognate.shape[0]

    # 发明人数量
    metrics['发明人数量'] = filter_value['发明人数量'].sum()
    metrics['发明人数量_去重'] = filter_value['发明人'].str.split(';').explode().str.strip().nunique()
    metrics['发明人数量_平均'] = metrics['发明人数量_去重'] / year_interval(filter_value, '发明人')

    # 权利要求数量和引证科技文献数量
    metrics['权利要求数量'] = filter_value['权利要求数量'].sum()
    metrics['引证科技文献数量'] = filter_value['引证科技文献'].count()  # 80%空,格式不统一
    
    # 有效专利数量和有效比例
    metrics['有效数量'] = (filter_value['专利有效性'] == '有效').sum()
    metrics['有效_比例'] = metrics['有效数量'] / filter_value.shape[0]

    # 发明授权和发明申请专利数量
    metrics['发明授权'] = (filter_value['专利类型'] == '发明授权').sum()
    metrics['发明申请'] = (filter_value['专利类型'] == '发明申请').sum()
    metrics['发明数量'] = ((filter_value['专利类型'] == '发明授权') | (filter_value['专利类型'] == '发明申请')).sum()

    metrics['实用新型']=(filter_value['专利类型'] == '实用新型').sum()
    metrics['外观设计']=(filter_value['专利类型'] == '外观设计').sum()

    copatent = (filter_value['申请人'].str.count(';') + 1 > 1)
    metrics['invention'] = (~copatent & ((filter_value['专利类型'] == '发明授权') | (filter_value['专利类型'] == '发明申请'))).sum()
    metrics['shiyong'] = (~copatent & (filter_value['专利类型'] == '实用新型')).sum()
    metrics['waiguan'] = (~copatent & (filter_value['专利类型'] == '外观设计')).sum()
    
    # 联合申请
    metrics['copatent_invention'] = (copatent & ((filter_value['专利类型'] == '发明授权') | (filter_value['专利类型'] == '发明申请'))).sum()
    metrics['copatent_shiyong'] = (copatent & (filter_value['专利类型'] == '实用新型')).sum()
    metrics['copatent_waiguan'] = (copatent & (filter_value['专利类型'] == '外观设计')).sum()

    return metrics

chinese_to_arabic = {
    '零': '0', '一': '1', '二': '2', '三': '3', '四': '4',
    '五': '5', '六': '6', '七': '7', '八': '8', '九': '9'
}

# 1. 剔除异常符号
def remove_unwanted_symbols(name):
    # 定义允许的字符：中文、英文、数字和常见符号（不想删除的符号）
    allowed_characters = r'[^\w\s\(\)（）【】「」『』]'
    # 删除所有不允许的符号
    return re.sub(allowed_characters, '', name)

# 2. 将各种括号转换成英文括号
def convert_brackets(name):
    # 替换中文和全角括号为标准英文括号
    return name.replace('（', '(').replace('）', ')') \
               .replace('【', '[').replace('】', ']') \
               .replace('「', '[').replace('」', ']') \
               .replace('『', '[').replace('』', ']')

# 3. 将汉字数字替换成阿拉伯数字
def replace_chinese_numbers(name):
    for ch_num, ar_num in chinese_to_arabic.items():
        name = name.replace(ch_num, ar_num)
    return name

# 4. 检查并调整括号匹配
def fix_unbalanced_brackets(name):
    left_count = name.count('(')
    right_count = name.count(')')
    
    # 如果左括号比右括号多，补齐右括号
    if left_count > right_count:
        name += ')' * (left_count - right_count)
    # 如果右括号比左括号多，补齐左括号
    elif right_count > left_count:
        name = '(' * (right_count - left_count) + name
    
    return name

# 5. 移除前面的数字和数字开头的部分
def remove_leading_numbers(name): 
    return re.sub(r'^\d+.*?(?=\D)', '', name).strip()  # re.sub(r'^\d+.*? ', '', name) 移除开头的数字直到第一个非数字字符，并去掉前后空格

    
# 综合清洗函数
def clean_fullname(name):
    if not isinstance(name, str):
        return name
    name = remove_unwanted_symbols(name)  # 剔除异常符号
    name = convert_brackets(name)  # 将括号转换成英文括号
    name = replace_chinese_numbers(name)  # 替换汉字数字为阿拉伯数字
    name = fix_unbalanced_brackets(name)  # 调整不平衡的括号
    name = remove_leading_numbers(name)  # 移除前面的数字
    return name.upper() # 转换为大写并去掉前后空格

words_to_remove = [
    '有限', '责任', '股份', '公司', '厂', '省', '市', '区', '县',
    '回族自治区', '壮族自治区', '维吾尔自治区', '自治区'
]
# words_to_remove = [
#     '有限', '责任', '股份', '集团', '总公司', '分公司', '公司', '总院', '分院', '总部', '分部', '总厂', '厂', 
#     '省', '市', '区', '县', '回族自治区', '壮族自治区', '维吾尔自治区', '自治区'
# ]


# 1. 剔除特定词汇
def remove_specific_words(name):
    # 构建正则表达式，匹配列表中的词汇
    pattern = '|'.join(words_to_remove)
    # 替换这些词汇为空
    return re.sub(pattern, '', name)

# 2. 剔除括号和括号内的内容
def remove_brackets(name):
    # 删除所有括号和其中的内容
    return re.sub(r'\(.*?\)', '', name)

def remove_outer_brackets(name):
    # 删除括号但保留其中的内容
    return re.sub(r'[()]', '', name)

# 综合清洗得到企业简称
def clean_short_name(name):
    if not isinstance(name, str):
        return name
    name = remove_specific_words(name)  # 剔除特定词汇
    name = remove_brackets(name)  # 剔除括号及其中内容
    return name.strip()  # 去除首尾空格

In [8]:
patent_tax_gx=pd.read_csv('X:\Downloads\Patent\patent_tax_gx.csv')
patent_tax_gx.shape#1432848+29874,32

C:\Users\Prime\AppData\Local\Temp\ipykernel_75944\659431814.py:1: DtypeWarning: Columns (18,19,22) have mixed types. Specify dtype option on import or set low_memory=False.
  patent_tax_gx=pd.read_csv('X:\Downloads\Patent\patent_tax_gx.csv')


(1462722, 32)

In [4]:
patent_tax_gx.columns

Index(['序号', '标题 (中文)', '申请人', '申请日', '申请号', '公开（公告）号', '公开（公告）日', '专利类型',
       '专利有效性', '公开国别', 'IPC主分类', 'IPC', '申请人类型', '发明人', '发明人数量', '简单同族',
       '扩展同族', '简单同族个数', '引证专利', '引证科技文献', '引证次数', '被引证次数', '引证申请人', '被引证申请人',
       '申请人数量', '许可次数', '转让次数', '权利要求数量', '当前法律状态', '当前权利人', '工商统一社会信用代码',
       'table_name'],
      dtype='object')

In [9]:
patent_tax_gx_1=pd.read_csv('data\patent_data_2019_202410.csv',usecols=['序号', '标题 (中文)', '申请人', '申请日', '申请号', '公开（公告）号', '公开（公告）日', '专利类型',
       '专利有效性', '公开国别', 'IPC主分类', 'IPC', '申请人类型', '发明人', '发明人数量', '简单同族',
       '扩展同族', '简单同族个数', '引证专利', '引证科技文献', '引证次数', '被引证次数', '引证申请人', '被引证申请人',
       '申请人数量', '许可次数', '转让次数', '权利要求数量', '当前法律状态', '当前权利人', '工商统一社会信用代码'])
patent_tax_gx_1['table_name']='patent202410'
patent_tax_gx=pd.concat([patent_tax_gx,patent_tax_gx_1])
patent_tax_gx.shape#2163441

(2150982, 32)

In [10]:
patent_tax_gx=pd.concat([patent_tax_gx,pd.read_csv('X:\Downloads\Patent\patent_tax_gx_2.csv'),pd.read_csv('X:\Downloads\Patent\patent_tax_gx_3.csv'),pd.read_csv('X:\Downloads\Patent\patent_tax_gx_4.csv')])
patent_tax_gx.shape

(2173070, 32)

In [11]:
patent_tax_gx['申请日']=pd.to_datetime(patent_tax_gx['申请日'])
patent_tax_gx['公开（公告）日']=pd.to_datetime(patent_tax_gx['公开（公告）日'])
patent_tax_gx['申请日'].describe(),patent_tax_gx['公开（公告）日'].describe()

(count                          2173070
 mean     2014-09-28 09:05:27.130558464
 min                1985-04-25 00:00:00
 25%                2012-05-30 00:00:00
 50%                2015-05-29 00:00:00
 75%                2017-09-29 00:00:00
 max                2019-12-31 00:00:00
 Name: 申请日, dtype: object,
 count                          2173070
 mean     2015-11-25 09:50:16.616676864
 min                1986-03-12 00:00:00
 25%                2013-05-08 00:00:00
 50%                2016-05-18 00:00:00
 75%                2018-10-19 00:00:00
 max                2024-10-01 00:00:00
 Name: 公开（公告）日, dtype: object)

In [12]:
patent_tax_gx=patent_tax_gx.sort_values(['申请日','公开（公告）日']).drop_duplicates(['申请号','公开（公告）号'],keep='first').reset_index(drop=True)
patent_tax_gx.shape#1198466,1208568,1896828,1896890

(1905166, 32)

In [14]:
patent_tax_gx.申请号.nunique()#1698733,1698680,1698733->1703961

1703961

In [38]:
tax_gx_patent_bridge=pd.read_csv('data\\tax_gx_patent_bridge_2019.csv')
tax_gx_patent_bridge=patent_tax_gx_1[~patent_tax_gx_1['公开（公告）号'].isin(patent_tax_gx['公开（公告）号'])]
tax_gx_patent_bridge.to_csv('X:\Downloads\Patent\\tax_gx_patent_bridge_2019_2.csv')
tax_gx_patent_bridge.shape

(12603, 5)

In [116]:
tax_gx_patent_bridge=pd.read_csv('X:\Downloads\Patent\\tax_gx_patent_bridge_2410.csv',index_col=0)
(~tax_gx_patent_bridge['申请号'].isin(patent_co['申请号'])).sum(),(~tax_gx_patent_bridge['企业简称'].isin(patent_co['企业简称'])).sum()

(62, 55)

In [117]:
tax_gx_patent_bridge=tax_gx_patent_bridge[~tax_gx_patent_bridge['申请号'].isin(patent_co['申请号'])|~tax_gx_patent_bridge['企业简称'].isin(patent_co['企业简称'])]
tax_gx_patent_bridge['申请号'].nunique()

53

In [129]:
tax_gx_patent_bridge.to_csv('X:\Downloads\Patent\\tax_gx_patent_bridge_2019_3.csv')
tax_gx_patent_bridge.table_name.value_counts()

Object `tax_gx_patent_bridge.to_csv('X:\Downloads\Patent\\tax_gx_patent_bridge_2019_3.csv')` not found.


table_name
patent202210    32
patent202408    14
patent202404    12
patent202407     4
Name: count, dtype: int64

In [43]:
patent_tax_gx_companie=pd.read_csv('data\\tax_gx_year_sort_match.csv',index_col=0) 
patent_tax_gx_companie=patent_tax_gx_companie[patent_tax_gx_companie['观察年']!=2019].reset_index(drop=True)
patent_tax_gx_companie.企业简称.nunique()#35359,35438

35438

In [24]:
patent_tax_gx_companie.shape

(553155, 20)

In [17]:
# patent_applicants=patent_tax_gx['申请人'].str.split(';').explode().str.strip()
# patent_applicants_short=patent_applicants.apply(clean_fullname).apply(clean_short_name)
# mask=patent_applicants_short.isin(combined_tax_gx_companies)
# patent_all_7 = patent_tax_gx.loc[mask[mask].index]

patent_tax_gx['申请年']=patent_tax_gx['申请日'].dt.year
patent_tax_gx['企业名称']=patent_tax_gx['申请人'].str.split(';')

patent_co=patent_tax_gx.explode(['企业名称'])

patent_co['企业名称']=patent_co['企业名称'].str.strip()
patent_co['企业简称']=patent_co['企业名称'].apply(clean_fullname).apply(clean_short_name).replace('',pd.NA)

patent_co=patent_co[patent_co['企业简称'].isin(patent_tax_gx_companie.企业简称)].sort_values('申请年').reset_index(drop=True)
patent_co.shape,patent_co['企业简称'].nunique(),patent_co.申请号.nunique()#((1306185, 37), 8451, 1077022)((2009505, 37), 35359, 1698733)

((2018670, 35), 35438, 1703961)

In [37]:
patent_co['企业名称'].nunique(),patent_co['申请号'].nunique()#35359,37255,37283  ->1703961

(37376, 1703961)

In [19]:
patent_co.to_csv('data\patent_co_tax_gx_2018_37k_2019.csv',encoding='utf-8')

In [20]:
patent_co['技术大类']=patent_co.IPC主分类.str.strip().str[:3]
patent_co['引证专利_A'] = patent_co['引证专利'].fillna('').str.split(';').apply(lambda x: [i.strip() for i in x if i.strip()])

In [30]:
n = [6, 5, 4, 3, 2] 
m1 = [5, 4, 3, 2, 1] 
m2 = [4, 3, 2, 1, 0]
tuple_list =[]
for ni in n:
    tuple_list.append((ni, m1[n.index(ni)])) 
    tuple_list.append((ni, m2[n.index(ni)])) 
tuple_list

[(6, 5),
 (6, 4),
 (5, 4),
 (5, 3),
 (4, 3),
 (4, 2),
 (3, 2),
 (3, 1),
 (2, 1),
 (2, 0)]

In [ ]:
%%time
knowledge_year=5
df_res=pd.DataFrame()

for year in range(2004,2019+1):
    year_res=[]
    patent_value=patent_co[patent_co['申请年'] <=  year]
    max_date=pd.to_datetime(year, format='%Y')+ pd.offsets.YearEnd()

    for co in sorted(combined_tax_gx_companies):
        filter_patent= patent_value[patent_value['企业简称']==co]
        if filter_patent.empty:
            continue

        co_metrics ={'year':year,'企业简称':co,'qymc':map_name_co.get(co),'专利申请人':';'.join(filter_patent['企业名称'].dropna().unique())}
        co_metrics['知识宽度'] = process_ipc(filter_patent) # 累积量
        mask_years = filter_patent['申请年'].between(year - knowledge_year, year - 1)
        co_metrics['patent_width'] = process_ipc(filter_patent[mask_years]) 
        tech_category =  filter_patent.loc[mask_years,'技术大类'].dropna().to_list()
        past_citations = filter_patent.loc[mask_years, '引证专利_A'].explode().dropna()#.str.strip()
        past_patents = filter_patent.loc[filter_patent['申请年'] != year, '公开（公告）号'].dropna().str.strip()
        existing_knowledge = set(past_patents) | set(past_citations)

        for m,n in tuple_list:
            suffix=f'{m,n}'
            M=m*365
            N=n*365
            
            filter_value_date=filter_patent[filter_patent.申请日>=max_date-pd.Timedelta(days=N)]
            if filter_value_date.empty:
                continue
                
            mask=filter_value_date.申请日 <  max_date-pd.Timedelta(days=(N - M))
            tech_category_value = filter_value_date[mask]
            base_category_value = filter_value_date[~mask]
            
            innovation_patent_num=len(set(tech_category_value.技术大类)^set(base_category_value.技术大类)) 
            co_metrics[f'innovation_patent_num_ipc_{suffix}']=innovation_patent_num
            co_metrics[f'ipc_innovationc_{suffix}']=innovation_patent_num / len(filter_value_date)
        
        filter_value = filter_patent[filter_patent['申请年'] ==  year]
        if not filter_value.empty:           
            co_metrics.update(calculate_metrics(filter_value))   # 计算指标
       
            explorative_percentage = filter_value['引证专利_A'].apply(lambda x:len(set(x) & set(existing_knowledge)) / len(x) if (isinstance(x, list) and len(x)) else 0)
            co_metrics['explorative'] = (explorative_percentage > 0.8).sum() / len(explorative_percentage)
            co_metrics['jianjin_patent_num'] = filter_value['技术大类'].isin(tech_category).sum()
            
        year_res.append(co_metrics)

    df_res = pd.concat([df_res, pd.DataFrame(year_res)], ignore_index=True) 

    print(year,len(year_res),patent_value['企业简称'].nunique(),df_res.shape)


2004 786 786 (786, 57)
2005 1045 1045 (1831, 57)


In [102]:
def patent_width(row,year=5):
    filter_patent=patent_co[patent_co['申请年'].between(row.year - year, row.year)&(patent_co['企业简称'] == row['企业简称'])]
    return process_ipc(filter_patent)
    
#df_res['patent_width']=df_res.apply(patent_width, axis=1)
df_res

,year,企业简称,qymc,专利申请人,知识宽度,patent_width,tech_category,existing_knowledge,"innovation_patent_num_ipc_(6, 5)","ipc_innovationc_(6, 5)",...,waiguan,copatent_invention,copatent_shiyong,copatent_waiguan,"innovation_patent_num_ipc_(2, 0)","ipc_innovationc_(2, 0)",申请年_x,explorative,申请年_y,jianjin_patent_num
0,2004,UT斯达康通讯,UT斯达康通讯有限公司,UT斯达康通讯有限公司,0.000000,1.000000,,,1.0,0.250000,...,0.0,0.0,0.0,0.0,NaN,NaN,2004.0,0.0,2004.0,0.0
1,2004,万达信息,万达信息股份有限公司,万达信息股份有限公司,0.444444,0.444444,G06;H04,CN100411380C;CN1201190A;CN1210409A;CN1217265C;...,2.0,0.333333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2004,上海万兴生物制药,上海万兴生物制药有限公司,上海万兴生物制药有限公司,0.447347,0.447347,C12,CN100335622C;CN100339478C;CN1182155C;CN1194308...,1.0,0.250000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2004,上海东升新材料,上海东升新材料有限公司,上海东升新材料有限公司,0.320000,1.000000,,,1.0,1.000000,...,0.0,1.0,0.0,0.0,NaN,NaN,2004.0,0.0,2004.0,0.0
4,2004,上海东富龙科技,上海东富龙科技股份有限公司;上海东富龙科技有限公司,上海东富龙科技有限公司,0.493827,0.375000,F16;F26,CN1327149C;CN1480670A;CN2146614Y;CN2148219Y;CN...,3.0,0.500000,...,0.0,0.0,0.0,0.0,NaN,NaN,2004.0,0.0,2004.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74517,2019,龙建路桥,龙建路桥股份有限公司,龙建路桥股份有限公司,0.856684,0.863772,A01;B07;B09;B28;B65;E01;E02;E04;E21;G01,CN101033597B;CN101581065B;CN101586336A;CN10159...,13.0,0.361111,...,0.0,5.0,6.0,1.0,NaN,NaN,2019.0,0.0,2019.0,9.0
74518,2019,龙星化工,龙星化工股份有限公司,龙星化工股份有限公司,0.860021,0.774691,B01;B03;B65;C01;C09;F17;F23;G05,CN101279738A;CN101891976A;CN102286221B;CN10297...,8.0,0.571429,...,0.0,0.0,0.0,0.0,NaN,NaN,2019.0,0.0,2019.0,2.0
74519,2019,龙福环能科技,龙福环能科技股份有限公司,龙福环能科技股份有限公司,0.780104,0.822936,A23;A47;B08;B32;B65;D01;D02;D06;F24,CN100999845A;CN101025040A;CN101302671A;CN10130...,10.0,0.357143,...,0.0,0.0,0.0,0.0,NaN,NaN,2019.0,0.0,2019.0,5.0
74520,2019,龙芯中科技术,龙芯中科技术有限公司,龙芯中科技术有限公司;龙芯中科技术股份有限公司;龙芯中科(山西)技术有限公司;龙芯中科(成都...,0.398258,0.407921,G01;G05;G06;G09;G11;H01;H03;H04;H05,AU5538686A;CA2439633A1;CN100354851C;CN10057824...,10.0,0.024155,...,0.0,0.0,0.0,0.0,1.0,0.25,2019.0,0.0,2019.0,67.0


In [97]:
df_res.columns

Index(['year', '企业简称', 'qymc', '专利申请人', '知识宽度', 'patent_width',
       'tech_category', 'existing_knowledge',
       'innovation_patent_num_ipc_(6, 5)', 'ipc_innovationc_(6, 5)',
       'innovation_patent_num_ipc_(6, 4)', 'ipc_innovationc_(6, 4)',
       'innovation_patent_num_ipc_(5, 4)', 'ipc_innovationc_(5, 4)',
       'innovation_patent_num_ipc_(5, 3)', 'ipc_innovationc_(5, 3)',
       'innovation_patent_num_ipc_(4, 3)', 'ipc_innovationc_(4, 3)',
       'innovation_patent_num_ipc_(4, 2)', 'ipc_innovationc_(4, 2)',
       'innovation_patent_num_ipc_(3, 2)', 'ipc_innovationc_(3, 2)',
       'innovation_patent_num_ipc_(3, 1)', 'ipc_innovationc_(3, 1)',
       'innovation_patent_num_ipc_(2, 1)', 'ipc_innovationc_(2, 1)',
       'Application_num', 'IPC主分类_去重', 'IPC主分类_平均', 'IPC_个数_去重', 'IPC_个数_平均',
       '被引证次数', '引证数量', '引证数量_去重', '引证数量_平均', '简单同族个数', '简单同族_去重',
       '简单同族美日欧_比例', '发明人数量', '发明人数量_去重', '发明人数量_平均', '权利要求数量', '引证科技文献数量',
       '有效数量', '有效_比例', '发明授权', '发明申请', '实用新型', 

In [103]:
del df_res['申请年_x']
del df_res['申请年_y']

In [99]:
df_res.to_excel('data\\tax_gx_year.xlsx')

In [20]:
patent_tax_gx_companie['years'].unique()

array([2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016],
      dtype=int64)

In [22]:
patent_tax_gx_companie.columns

Index(['企业简称', 'qymc', '高新企业名称', 'match_gx', 'match_score', 'count_name_qymc',
       'years'],
      dtype='object')

In [29]:
patent_co.groupby('企业简称')['企业名称'].nunique().sort_values().tail(10)

企业简称
芯原微电子       4
优利德科技       4
佛山海天调味食品    4
东软集团        5
联化科技        5
中兴通讯        5
龙芯中科技术      5
中集车辆        5
光大环保能源      7
格力电器        9
Name: 企业名称, dtype: int64

In [31]:
patent_co[patent_co.企业简称=='格力电器']['企业名称'].unique()

array(['格力电器(重庆)有限公司', '格力电器(郑州)有限公司', '格力电器(合肥)有限公司', '格力电器(武汉)有限公司',
       '格力电器(芜湖)有限公司', '格力电器(石家庄)有限公司', '格力电器(杭州)有限公司', '格力电器(成都)有限公司',
       '格力电器(洛阳)有限公司'], dtype=object)

In [34]:
patent_tax_gx_companie[patent_tax_gx_companie.企业简称=='格力电器']['qymc'].unique()

array(['格力电器(合肥)有限公司', '格力电器(重庆)有限公司'], dtype=object)

In [42]:
patent_tax_gx_companie[patent_tax_gx_companie.企业简称=='格力电器']['高新企业名称'].str.split(';').explode().unique()

array(['格力电器（杭州）有限公司', '格力电器（合肥）有限公司', '格力电器（洛阳）有限公司', '格力电器（南京）有限公司',
       '格力电器（石家庄）有限公司', '格力电器（芜湖）有限公司', '格力电器（武汉）有限公司', '格力电器（郑州）有限公司',
       '格力电器（重庆）有限公司'], dtype=object)

In [63]:
patent_co.groupby('申请号')['申请日'].nunique().sort_values()

申请号
CN00100333.X        1
CN201621447659.2    1
CN201621447637.6    1
CN201621447592.2    1
CN201621447589.0    1
                   ..
CN200580047602.1    2
CN200810156871.7    2
CN201810004532.0    2
CN02122989.9        2
CN201780000686.6    2
Name: 申请日, Length: 1698680, dtype: int64

In [28]:
patent_co['申请日'].max()

Timestamp('2019-12-31 00:00:00')

In [27]:
patent_co.申请年.value_counts()

申请年
2018    231339
2017    225589
2016    218707
2015    197145
2019    177336
2013    176446
2014    175488
2012    163358
2011    129141
2010     94512
2009     71688
2008     49857
2007     37102
2006     25165
2005     16761
2004     10488
2003      7511
2002      4370
2001      2515
2000      1549
1999      1028
1998       501
1997       379
1996       201
1995       144
1994       130
1993        88
1992        46
1991        19
1987        19
1990        14
1988        13
1986        12
1989         6
1985         3
Name: count, dtype: int64

In [65]:
patent_co[patent_co.申请号=='CN200580047602.1']

,序号,标题 (中文),申请人,申请日,申请号,公开（公告）号,公开（公告）日,专利类型,专利有效性,公开国别,...,权利要求数量,当前法律状态,当前权利人,工商统一社会信用代码,table_name,申请年,企业名称,企业简称,技术大类,引证专利_A
36951,33058,扩频系统中窄带干扰消除的方法、装置,中兴通讯股份有限公司,2005-12-07,CN200580047602.1,CN100550666C,2009-10-14,发明授权,失效,中国,...,15.0,未缴年费,中兴通讯股份有限公司,9144030027939873X7,patent202210,2005,中兴通讯股份有限公司,中兴通讯,H04,"[CN1592125A, US6868114B2, US6760319B1]"
102999,47116,扩频系统中窄带干扰消除的方法、装置,中兴通讯股份有限公司,2007-06-14,CN200580047602.1,CN101112003A,2008-01-23,发明申请,失效,中国,...,1.0,未缴年费,中兴通讯股份有限公司,9144030027939873X7,patent202404,2007,中兴通讯股份有限公司,中兴通讯,H04,[]


In [84]:
clean_fullname('万向一二三股份公司')

'万向123股份公司'

In [31]:
import difflib

In [139]:
%%time
knowledge_year=5

for year in patent_tax_gx_companie.years.unique():
    max_date=pd.to_datetime(year, format='%Y')+ pd.offsets.YearEnd()
    patent_value=patent_co[patent_co['申请年'] <=  year]
    qymc_value=patent_tax_gx_companie[patent_tax_gx_companie.years == year]

    for index,row in qymc_value.iterrows():
        filter_patent= patent_value[patent_value['企业简称']==row['企业简称']]
        applicants=filter_patent['企业名称'].dropna().unique()
        match_applicant=difflib.get_close_matches(row['qymc'], applicants, n=1, cutoff=0.8)
        match_applicant = match_applicant[0] if match_applicant else None
        if row['count_name_qymc']>1:
            if match_applicant:
                filter_patent= patent_value[patent_value['企业名称']==match_applicant]
                applicants=[match_applicant]
            else:
                pass
                # match_applicant=difflib.get_close_matches(row['match_gx'], applicants, n=1, cutoff=0.8),axis=1)
        filter_patent= filter_patent.sort_values(['申请日','公开（公告）日']).drop_duplicates(['申请号','专利类型'],keep='last')#'申请日',
        if filter_patent.empty:
            continue

        co_metrics ={'专利申请人':';'.join(applicants),'match_applicant':match_applicant}
        
        co_metrics['知识宽度'] = process_ipc(filter_patent) # 累积量
        mask_years = filter_patent['申请年'].between(year - knowledge_year, year - 1)
        co_metrics['patent_width'] = process_ipc(filter_patent[mask_years]) 
        tech_category =  filter_patent.loc[mask_years,'技术大类'].dropna().to_list()
        past_citations = filter_patent.loc[mask_years, '引证专利_A'].explode().dropna()#.str.strip()
        past_patents = filter_patent.loc[filter_patent['申请年'] != year, '公开（公告）号'].dropna().str.strip()
        existing_knowledge = set(past_patents) | set(past_citations)

        co_metrics.update(calculate_metrics(filter_patent))

        for m,n in tuple_list:
            suffix=f'{m,n}'
            M=m*365
            N=n*365
            
            filter_value_date=filter_patent[filter_patent.申请日>=max_date-pd.Timedelta(days=N)]
            if filter_value_date.empty:
                continue
                
            mask=filter_value_date.申请日 <  max_date-pd.Timedelta(days=(N - M))
            tech_category_value = filter_value_date[mask]
            base_category_value = filter_value_date[~mask]
            
            innovation_patent_num=len(set(tech_category_value.技术大类)^set(base_category_value.技术大类)) 
            co_metrics[f'innovation_patent_num_ipc_{suffix}']=innovation_patent_num
            co_metrics[f'ipc_innovationc_{suffix}']=innovation_patent_num / len(filter_value_date)
        
        filter_value = filter_patent[filter_patent['申请年'] ==  year]
        if not filter_value.empty:           
            co_metrics.update({f'year_{k}': v for k, v in calculate_metrics(filter_value).items()})   # 计算指标
       
            explorative_percentage = filter_value['引证专利_A'].apply(lambda x:len(set(x) & set(existing_knowledge)) / len(x) if (isinstance(x, list) and len(x)) else 0)
            co_metrics['explorative'] = (explorative_percentage > 0.8).sum() / len(explorative_percentage)
            co_metrics['jianjin_patent_num'] = filter_value['技术大类'].isin(tech_category).sum()
            
        patent_tax_gx_companie.loc[index, co_metrics.keys()] = pd.Series(co_metrics) 

    print(year,patent_value['企业简称'].nunique(),qymc_value['企业简称'].nunique())# 4h

2007 6521 8591
2008 8766 16983
2009 11608 16983
2010 14532 18707
2011 17696 19755
2012 20849 20565
2013 23570 21035
2014 25780 16961
2015 28069 14948
2016 30497 14300
CPU times: total: 4h 1min 58s
Wall time: 4h 2min


In [140]:
patent_tax_gx_companie.columns

Index(['企业简称', 'qymc', '高新企业名称', 'match_gx', 'match_score', 'count_name_qymc',
       'years', '专利申请人', 'match_applicant', '知识宽度', 'patent_width',
       'Application_num', 'IPC主分类_去重', 'IPC主分类_平均', 'IPC_个数_去重', 'IPC_个数_平均',
       '被引证次数', '引证数量', '引证数量_去重', '引证数量_平均', '简单同族个数', '简单同族_去重',
       '简单同族美日欧_比例', '发明人数量', '发明人数量_去重', '发明人数量_平均', '权利要求数量', '引证科技文献数量',
       '有效数量', '有效_比例', '发明授权', '发明申请', '实用新型', '外观设计', 'invention', 'shiyong',
       'waiguan', 'copatent_invention', 'copatent_shiyong', 'copatent_waiguan',
       'innovation_patent_num_ipc_(6, 5)', 'ipc_innovationc_(6, 5)',
       'innovation_patent_num_ipc_(6, 4)', 'ipc_innovationc_(6, 4)',
       'innovation_patent_num_ipc_(5, 4)', 'ipc_innovationc_(5, 4)',
       'innovation_patent_num_ipc_(5, 3)', 'ipc_innovationc_(5, 3)',
       'innovation_patent_num_ipc_(4, 3)', 'ipc_innovationc_(4, 3)',
       'innovation_patent_num_ipc_(4, 2)', 'ipc_innovationc_(4, 2)',
       'innovation_patent_num_ipc_(3, 2)', 'ipc_innovation

In [32]:
patent_tax_gx_companie.index

RangeIndex(start=0, stop=553155, step=1)

In [ ]:
%%time
knowledge_year=5

for year in patent_tax_gx_companie.观察年.unique():
    max_date=pd.to_datetime(year, format='%Y')+ pd.offsets.YearEnd()
    patent_value=patent_co[patent_co['申请年'] <=  year]
    qymc_value=patent_tax_gx_companie[patent_tax_gx_companie.观察年 == year]

    for index,row in qymc_value.iterrows():
        filter_patent= patent_value[patent_value['企业简称']==row['企业简称']]
        applicants=filter_patent['企业名称'].dropna().unique()
        match_applicant=difflib.get_close_matches(row['qymc'], applicants, n=1, cutoff=0.8)
        match_applicant = match_applicant[0] if match_applicant else None
        if row['count_name_qymc']>1:
            if match_applicant:
                filter_patent= patent_value[patent_value['企业名称']==match_applicant]
                applicants=[match_applicant]
            else:
                pass
                # match_applicant=difflib.get_close_matches(row['match_gx'], applicants, n=1, cutoff=0.8),axis=1)
        filter_patent= filter_patent.sort_values(['申请日','公开（公告）日']).drop_duplicates(['申请号','专利类型'],keep='last')#'申请日',
        if filter_patent.empty:
            continue

        co_metrics ={'专利申请人':';'.join(applicants),'match_applicant':match_applicant}
        
        co_metrics['知识宽度'] = process_ipc(filter_patent) # 累积量
        mask_years = filter_patent['申请年'].between(year - knowledge_year -1, year - 1)
        co_metrics[f'前{knowledge_year}年知识宽度'] = process_ipc(filter_patent[mask_years]) 
        tech_category =  filter_patent.loc[mask_years,'技术大类'].dropna().to_list()
        past_citations = filter_patent.loc[mask_years, '引证专利_A'].explode().dropna()#.str.strip()
        past_patents = filter_patent.loc[filter_patent['申请年'] != year, '公开（公告）号'].dropna().str.strip()
        existing_knowledge = set(past_patents) | set(past_citations)

        for n in range(knowledge_year, 0, -1):
            mask_years = filter_patent['申请年'].between(year - n - 1, year - 1)
            co_metrics[f'前{n}年被引证次数']=filter_patent.loc[mask_years, '被引证次数'].sum()

        co_metrics.update(calculate_metrics(filter_patent))

        for m,n in tuple_list:
            suffix=f'{m,n}'
            M=m*365
            N=n*365
            
            filter_value_date=filter_patent[filter_patent.申请日>=max_date-pd.Timedelta(days=N)]
            if filter_value_date.empty:
                continue
                
            mask=filter_value_date.申请日 <  max_date-pd.Timedelta(days=(N - M))
            tech_category_value = filter_value_date[mask]
            base_category_value = filter_value_date[~mask]

            innovation_ipc =set(tech_category_value.技术大类)^set(base_category_value.技术大类)
            innovation_patent_num=filter_value_date.技术大类.isin(innovation_ipc).sum()
            co_metrics[f'突破性创新2_申请日_{suffix}']=innovation_patent_num
            co_metrics[f'突破性创新3_申请日_{suffix}']=len(innovation_ipc)#innovation_ipc_num 
            co_metrics[f'ipc_innovation_申请日_{suffix}']=innovation_patent_num / len(filter_value_date)
            #---
            mask= filter_value_date['公开（公告）日'] <  max_date-pd.Timedelta(days=(N - M))
            tech_category_value = filter_value_date[mask]
            base_category_value = filter_value_date[~mask]

            innovation_ipc =set(tech_category_value.技术大类)^set(base_category_value.技术大类)
            innovation_patent_num=filter_value_date.技术大类.isin(innovation_ipc).sum()
            co_metrics[f'突破性创新2_公告日_{suffix}']=innovation_patent_num
            co_metrics[f'突破性创新3_公告日_{suffix}']=len(innovation_ipc)#innovation_ipc_num  
            co_metrics[f'ipc_innovation_公告日_{suffix}']=innovation_patent_num / len(filter_value_date)
        
        filter_value = filter_patent[filter_patent['申请年'] ==  year]
        if not filter_value.empty:           
            co_metrics.update({f'year_{k}': v for k, v in calculate_metrics(filter_value).items()})   # 计算指标
       
            explorative_percentage = filter_value['引证专利_A'].apply(lambda x:len(set(x) & set(existing_knowledge)) / len(x) if (isinstance(x, list) and len(x)) else 0)
            co_metrics['explorative'] = (explorative_percentage > 0.8).sum() / len(explorative_percentage)
            co_metrics['jianjin_patent_num'] = filter_value['技术大类'].isin(tech_category).sum()
            
        co_metrics = {'patent_'+ key: value for key, value in co_metrics.items()}
        patent_tax_gx_companie.loc[index, co_metrics.keys()] = pd.Series(co_metrics) 

    print(year,patent_value['企业简称'].nunique())# 4h,125796,35438

In [42]:
patent_tax_gx_companie.columns.to_list()

['企业简称',
 'qymc',
 '高新企业名称',
 'tax_years',
 'count_name_qymc',
 'match_gx',
 'match_score',
 '火炬承担单位',
 'match_hj',
 '观察年',
 'isintax',
 '税调',
 'isinhj',
 '火炬',
 'treat_hj',
 'isingx',
 '高新',
 'treat_gx',
 'isinpt',
 '专利',
 'patent_专利申请人',
 'patent_match_applicant',
 'patent_知识宽度',
 'patent_前5知识宽度',
 'patent_前1年被引证次数',
 'patent_前2年被引证次数',
 'patent_前3年被引证次数',
 'patent_前4年被引证次数',
 'patent_前5年被引证次数',
 'patent_Application_num',
 'patent_IPC主分类_去重',
 'patent_IPC主分类_平均',
 'patent_IPC_个数_去重',
 'patent_IPC_个数_平均',
 'patent_被引证次数',
 'patent_被引证次数_平均',
 'patent_引证数量',
 'patent_引证数量_去重',
 'patent_引证数量_平均',
 'patent_简单同族个数',
 'patent_简单同族_去重',
 'patent_简单同族美日欧_比例',
 'patent_发明人数量',
 'patent_发明人数量_去重',
 'patent_发明人数量_平均',
 'patent_权利要求数量',
 'patent_引证科技文献数量',
 'patent_有效数量',
 'patent_有效_比例',
 'patent_发明授权',
 'patent_发明申请',
 'patent_发明数量',
 'patent_实用新型',
 'patent_外观设计',
 'patent_invention',
 'patent_shiyong',
 'patent_waiguan',
 'patent_copatent_invention',
 'patent_copatent_shiyong',
 'patent_copa

In [ ]:
patent_tax_gx_companie.to_csv('data\\税调与专利2018.csv', encoding="utf_8_sig")

In [145]:
hj=pd.read_excel('data/合并后火炬项目文件.xlsx',usecols=['承担单位','立项时间']).drop_duplicates().dropna(how='all')
hj['企业全称'] = hj['承担单位'].apply(clean_fullname)
hj['企业简称'] = hj['企业全称'].apply(clean_short_name).replace('',pd.NA)
hj

,承担单位,立项时间,企业全称,企业简称
0,北京华胜天成科技股份有限公司,2010,北京华胜天成科技股份有限公司,北京华胜天成科技
1,北京协成致远网络科技有限公司,2010,北京协成致远网络科技有限公司,北京协成致远网络科技
2,北京维深科技发展有限公司,2010,北京维深科技发展有限公司,北京维深科技发展
3,北京鼎普科技股份有限公司,2010,北京鼎普科技股份有限公司,北京鼎普科技
4,北京华诺恒网络科技有限公司,2010,北京华诺恒网络科技有限公司,北京华诺恒网络科技
...,...,...,...,...
27208,浙江航天神舟电控技术有限公司,2007,浙江航天神舟电控技术有限公司,浙江航天神舟电控技术
27210,山东爱通工业机器人科技有限公司,2007,山东爱通工业机器人科技有限公司,山东爱通工业机器人科技
27212,湖北叶威粮油机械有限公司,2007,湖北叶威粮油机械有限公司,湖北叶威粮油机械
27214,杭州锂软科技有限公司,2007,杭州锂软科技有限公司,杭州锂软科技


In [147]:
tmp=hj[hj['立项时间'] ==2008].copy()
tmp['立项时间']=2009
hj=pd.concat([hj,tmp])

In [148]:
patent_tax_gx_companie=patent_tax_gx_companie.merge(hj[['企业简称', '立项时间']], left_on=['企业简称', 'years'], right_on=['企业简称', '立项时间'], how='left', indicator=True)

In [149]:
patent_tax_gx_companie['isinhj'] =patent_tax_gx_companie['_merge'].apply(lambda x: 1 if x == 'both' else 0)
patent_tax_gx_companie.drop(columns=['_merge', '立项时间'], inplace=True)

In [150]:
sorted(hj['立项时间'].unique())

[2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014]

In [190]:
cumulative_hj_list = {}
for year in sorted(hj['立项时间'].unique()):
    if year not in cumulative_hj_list:
        cumulative_hj_list[year] = set(hj.loc[hj['立项时间'] <= year,'企业简称'].dropna())
# cumulative_hj_list[2009]=cumulative_hj_list[2008]#2008 if year==2009 else year
        
patent_tax_gx_companie['火炬'] = patent_tax_gx_companie.apply(lambda row: row['企业简称'] in cumulative_hj_list.get(row['years'], set()),axis=1).astype(int)

In [191]:
patent_tax_gx_companie['treat']=  patent_tax_gx_companie.groupby('企业简称')['火炬'].transform(lambda x: 0 if (x == 0).all() else 1)
patent_tax_gx_companie['treat'].sum()#17175, 1 if sum(x)>0  else 0

17175

In [154]:
patent_tax_gx_companie['企业简称'].isin(hj['企业简称'].dropna()).astype(int).sum()

17481

In [161]:
patent_tax_gx_companie['火炬承担单位']=patent_tax_gx_companie['企业简称'].map(hj.groupby('企业简称')['承担单位'].unique())
mask=patent_tax_gx_companie['火炬承担单位'].isna()
patent_tax_gx_companie.loc[~mask,'match_hj']=patent_tax_gx_companie[~mask].apply(lambda x: difflib.get_close_matches(x['qymc'], x['火炬承担单位'], n=1, cutoff=0.8),axis=1).explode()
patent_tax_gx_companie['火炬承担单位']=patent_tax_gx_companie['火炬承担单位'].dropna().apply(lambda x:';'.join(x))
patent_tax_gx_companie['match_hj'].count()

17409

In [193]:
patent_tax_gx_companie[(patent_tax_gx_companie['treat']==0)&(~patent_tax_gx_companie['match_hj'].isna())]

,企业简称,qymc,高新企业名称,match_gx,match_score,count_name_qymc,years,专利申请人,match_applicant,知识宽度,...,jianjin_patent_num,"innovation_patent_num_ipc_(2, 0)","ipc_innovationc_(2, 0)",isinhj,火炬,treat,火炬承担单位,match_hj,isingx,高新
2522,上海发凯化工,上海发凯化工有限公司,上海发凯化工有限公司,上海发凯化工有限公司,1.0,1,2008,NaN,NaN,NaN,...,NaN,NaN,NaN,0,0,0,上海发凯化工有限公司,上海发凯化工有限公司,0,0
2523,上海发凯化工,上海发凯化工有限公司,上海发凯化工有限公司,上海发凯化工有限公司,1.0,1,2009,NaN,NaN,NaN,...,NaN,NaN,NaN,0,0,0,上海发凯化工有限公司,上海发凯化工有限公司,0,0
2524,上海发凯化工,上海发凯化工有限公司,上海发凯化工有限公司,上海发凯化工有限公司,1.0,1,2011,NaN,NaN,NaN,...,NaN,NaN,NaN,0,0,0,上海发凯化工有限公司,上海发凯化工有限公司,0,0
2525,上海发凯化工,上海发凯化工有限公司,上海发凯化工有限公司,上海发凯化工有限公司,1.0,1,2012,NaN,NaN,NaN,...,NaN,NaN,NaN,0,0,0,上海发凯化工有限公司,上海发凯化工有限公司,0,0
2526,上海发凯化工,上海发凯化工有限公司,上海发凯化工有限公司,上海发凯化工有限公司,1.0,1,2013,NaN,NaN,NaN,...,NaN,NaN,NaN,0,0,0,上海发凯化工有限公司,上海发凯化工有限公司,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159818,铜陵恒兴化工,铜陵市恒兴化工有限责任公司,铜陵市恒兴化工有限责任公司,铜陵市恒兴化工有限责任公司,1.0,1,2008,NaN,NaN,NaN,...,NaN,NaN,NaN,0,0,0,铜陵市恒兴化工有限责任公司,铜陵市恒兴化工有限责任公司,0,0
159819,铜陵恒兴化工,铜陵市恒兴化工有限责任公司,铜陵市恒兴化工有限责任公司,铜陵市恒兴化工有限责任公司,1.0,1,2009,NaN,NaN,NaN,...,NaN,NaN,NaN,0,0,0,铜陵市恒兴化工有限责任公司,铜陵市恒兴化工有限责任公司,0,0
159820,铜陵恒兴化工,铜陵市恒兴化工有限责任公司,铜陵市恒兴化工有限责任公司,铜陵市恒兴化工有限责任公司,1.0,1,2010,NaN,NaN,NaN,...,NaN,NaN,NaN,0,0,0,铜陵市恒兴化工有限责任公司,铜陵市恒兴化工有限责任公司,0,0
159821,铜陵恒兴化工,铜陵市恒兴化工有限责任公司,铜陵市恒兴化工有限责任公司,铜陵市恒兴化工有限责任公司,1.0,1,2011,铜陵市恒兴化工有限责任公司,铜陵市恒兴化工有限责任公司,0.444444,...,0.0,NaN,NaN,0,0,0,铜陵市恒兴化工有限责任公司,铜陵市恒兴化工有限责任公司,0,0


In [175]:
hj[hj['承担单位']=='上海发凯化工有限公司']#2014

,承担单位,立项时间,企业全称,企业简称
7276,上海发凯化工有限公司,2014,上海发凯化工有限公司,上海发凯化工


In [201]:
patent_tax_gx_companie[patent_tax_gx_companie['企业简称']=='上海发凯化工']

,企业简称,qymc,高新企业名称,match_gx,match_score,count_name_qymc,years,专利申请人,match_applicant,知识宽度,...,jianjin_patent_num,"innovation_patent_num_ipc_(2, 0)","ipc_innovationc_(2, 0)",isinhj,火炬,treat,火炬承担单位,match_hj,isingx,高新
2522,上海发凯化工,上海发凯化工有限公司,上海发凯化工有限公司,上海发凯化工有限公司,1.0,1,2008,NaN,NaN,NaN,...,NaN,NaN,NaN,0,0,0,上海发凯化工有限公司,上海发凯化工有限公司,0,0
2523,上海发凯化工,上海发凯化工有限公司,上海发凯化工有限公司,上海发凯化工有限公司,1.0,1,2009,NaN,NaN,NaN,...,NaN,NaN,NaN,0,0,0,上海发凯化工有限公司,上海发凯化工有限公司,0,0
2524,上海发凯化工,上海发凯化工有限公司,上海发凯化工有限公司,上海发凯化工有限公司,1.0,1,2011,NaN,NaN,NaN,...,NaN,NaN,NaN,0,0,0,上海发凯化工有限公司,上海发凯化工有限公司,0,0
2525,上海发凯化工,上海发凯化工有限公司,上海发凯化工有限公司,上海发凯化工有限公司,1.0,1,2012,NaN,NaN,NaN,...,NaN,NaN,NaN,0,0,0,上海发凯化工有限公司,上海发凯化工有限公司,0,0
2526,上海发凯化工,上海发凯化工有限公司,上海发凯化工有限公司,上海发凯化工有限公司,1.0,1,2013,NaN,NaN,NaN,...,NaN,NaN,NaN,0,0,0,上海发凯化工有限公司,上海发凯化工有限公司,0,0


In [197]:
'上海发凯化工' in cumulative_hj_list[2014]

True

In [200]:
hj[hj['企业简称']=='铜陵恒兴化工']#2011,未来是

,承担单位,立项时间,企业全称,企业简称
6722,铜陵市恒兴化工有限责任公司,2014,铜陵市恒兴化工有限责任公司,铜陵恒兴化工


In [196]:
patent_tax_gx_companie.years.unique()

array([2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016],
      dtype=int64)

In [72]:
gx=pd.read_csv('data\高新技术企业基本信息_202408.csv',usecols=['企业名称','核准日期']).drop_duplicates().dropna(how='all')
gx.核准日期=pd.to_datetime(gx.核准日期)
gx['核准时间']=gx.核准日期.dt.year
gx['企业全称'] = gx['企业名称'].apply(clean_fullname)
gx['企业简称'] = gx['企业全称'].apply(clean_short_name).replace('',pd.NA)
gx

,企业名称,核准日期,企业全称,企业简称,核准时间
0,《健康时报》社有限责任公司,2024-07-18,健康时报社有限责任公司,健康时报社,2024.0
1,《全国新书目》杂志有限责任公司,2023-07-11,全国新书目杂志有限责任公司,全国新书目杂志,2023.0
2,《人民论坛》杂志社,2022-08-22,人民论坛杂志社,人民论坛杂志社,2022.0
3,《文化中国》网络科技（上海）有限公司,2019-02-21,文化中国网络科技(上海)有限公司,文化中国网络科技,2019.0
4,《中国城市报》社有限公司,2022-05-30,中国城市报社有限公司,中国城报社,2022.0
...,...,...,...,...,...
609590,柞水县科技投资发展有限公司,2021-09-10,柞水县科技投资发展有限公司,柞水科技投资发展,2021.0
609591,柞水县龙腾生态养殖有限公司,2021-01-26,柞水县龙腾生态养殖有限公司,柞水龙腾生态养殖,2021.0
609592,柞水县世纪生态农业有限公司,2021-03-12,柞水县世纪生态农业有限公司,柞水世纪生态农业,2021.0
609593,座头鲸（上海）信息技术有限公司,2015-01-07,座头鲸(上海)信息技术有限公司,座头鲸信息技术,2015.0


In [163]:
gx['核准时间'].isna().sum()

432

In [162]:
patent_tax_gx_companie=patent_tax_gx_companie.merge(gx[['企业简称', '核准时间']], left_on=['企业简称', 'years'], right_on=['企业简称', '核准时间'], how='left', indicator=True)
patent_tax_gx_companie['isingx'] =patent_tax_gx_companie['_merge'].apply(lambda x: 1 if x == 'both' else 0)
patent_tax_gx_companie.drop(columns=['_merge', '核准时间'], inplace=True)

In [164]:
cumulative_gx_list = {}
for year in sorted(gx['核准时间'].dropna().unique().astype(int)):
    if year not in cumulative_gx_list:
        cumulative_gx_list[year] = set(gx[gx['核准时间'] <= year]['企业简称'])
        
patent_tax_gx_companie['高新'] = patent_tax_gx_companie.apply(lambda row: row['企业简称'] in cumulative_gx_list.get(row['years'], set()), axis=1).astype(int)

In [167]:
patent_tax_gx_companie.shape,pd.read_csv('data\\tax_gx_year_sort_match.csv',index_col=0).shape

((169190, 98), (169163, 7))

In [170]:
patent_tax_gx_companie.to_csv('data\\税调与专利.csv', encoding="utf_8_sig")

In [61]:
patent_value.shape,patent_value.申请号.nunique()

((107275, 37), 90256)

In [65]:
qymc_gx_co=pd.read_excel('data\\税调&高新企业的税调的qymc.xlsx',index_col=0)
qymc_gx_co.shape

(109708, 2)

In [ ]:
difflib.get_close_matches(clean_fullname(row['qymc']), applicants, n=1, cutoff=0.8)

In [111]:
map_name_co=qymc_gx_co.groupby('企业简称')['qymc'].apply(lambda x:';'.join(set(x)))
# df_res['qymc']=df_res['企业简称'].map(map_name_co)
count_name_co=qymc_gx_co.groupby('企业简称')['qymc'].nunique().sort_values()
count_name_co.tail(20)

企业简称
SEW传动设备     4
重庆钢铁        4
今麦郎饮品       4
光宝电子        4
液化空气        4
建滔化工        4
光大环保能源      4
泰科电子        4
华锐风电科技      5
可口可乐饮料      5
泰山石膏        5
永丰余纸业       5
金红叶纸业       5
维达纸业        5
华润水泥        8
科技          9
燕京啤酒       10
青岛啤酒       11
华新水泥       15
华润雪花啤酒     25
Name: qymc, dtype: int64

In [115]:
df_res.loc[df_res['企业简称']=='华新水泥',['企业简称','qymc','专利申请人']]

,企业简称,qymc,专利申请人
206,华新水泥,华新水泥(迪庆)有限公司;华新水泥(道县)有限公司;华新水泥(昭通)有限公司;华新水泥(长阳...,华新水泥股份有限公司
1064,华新水泥,华新水泥(迪庆)有限公司;华新水泥(道县)有限公司;华新水泥(昭通)有限公司;华新水泥(长阳...,华新水泥股份有限公司
2200,华新水泥,华新水泥(迪庆)有限公司;华新水泥(道县)有限公司;华新水泥(昭通)有限公司;华新水泥(长阳...,华新水泥股份有限公司
3675,华新水泥,华新水泥(迪庆)有限公司;华新水泥(道县)有限公司;华新水泥(昭通)有限公司;华新水泥(长阳...,华新水泥股份有限公司
5603,华新水泥,华新水泥(迪庆)有限公司;华新水泥(道县)有限公司;华新水泥(昭通)有限公司;华新水泥(长阳...,华新水泥股份有限公司
8161,华新水泥,华新水泥(迪庆)有限公司;华新水泥(道县)有限公司;华新水泥(昭通)有限公司;华新水泥(长阳...,华新水泥股份有限公司
11386,华新水泥,华新水泥(迪庆)有限公司;华新水泥(道县)有限公司;华新水泥(昭通)有限公司;华新水泥(长阳...,华新水泥股份有限公司
15333,华新水泥,华新水泥(迪庆)有限公司;华新水泥(道县)有限公司;华新水泥(昭通)有限公司;华新水泥(长阳...,华新水泥股份有限公司
19977,华新水泥,华新水泥(迪庆)有限公司;华新水泥(道县)有限公司;华新水泥(昭通)有限公司;华新水泥(长阳...,华新水泥股份有限公司
25286,华新水泥,华新水泥(迪庆)有限公司;华新水泥(道县)有限公司;华新水泥(昭通)有限公司;华新水泥(长阳...,华新水泥股份有限公司


In [72]:
patent_co.loc[patent_co.企业简称=='华新水泥',['企业简称','企业名称','申请人']].drop_duplicates()

,企业简称,企业名称,申请人
2027,华新水泥,华新水泥股份有限公司,华新水泥股份有限公司
351110,华新水泥,华新水泥股份有限公司,华新水泥股份有限公司; 华新环境工程有限公司; 华新水泥(黄石)装备制造有限公司
351156,华新水泥,华新水泥股份有限公司,华新环境工程有限公司; 华新水泥股份有限公司; 华新水泥(黄石)装备制造有限公司
351465,华新水泥,华新水泥股份有限公司,华新水泥(黄石)装备制造有限公司; 华新水泥股份有限公司; 华新环境工程有限公司
368436,华新水泥,华新水泥股份有限公司,华新环境工程有限公司; 华新水泥股份有限公司
737485,华新水泥,华新水泥股份有限公司,华新水泥股份有限公司; 华新水泥(黄石)装备制造有限公司
850326,华新水泥,华新水泥股份有限公司,华新水泥(黄石)装备制造有限公司; 华新水泥股份有限公司
926818,华新水泥,华新水泥股份有限公司,华新水泥股份有限公司; 华新环境工程有限公司
979274,华新水泥,华新水泥股份有限公司,华新新型建材(武汉)有限公司; 华新水泥股份有限公司
1004122,华新水泥,华新水泥股份有限公司,重庆齿轮箱有限责任公司; 华新水泥股份有限公司
